In [38]:
import numpy as np
import pandas as pd

import sqlalchemy

import requests
import bs4
from glob import glob
import youtube_dl
import librosa

In [2]:
genres = {
    'country': 'https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S', 
    'jazz': 'https://www.youtube.com/playlist?list=PL8F6B0753B2CCA128', 
    'hip_hop': 'https://www.youtube.com/playlist?list=PLAPo1R_GVX4IZGbDvUH60bOwIOnZplZzM', 
    'classical': 'https://www.youtube.com/playlist?list=PLRb-5mC4V_Lop8KLXqSqMv4_mqw5M9jjW', 
    'metal': 'https://www.youtube.com/playlist?list=PLfY-m4YMsF-OM1zG80pMguej_Ufm8t0VC', 
    'electronic': 'https://www.youtube.com/playlist?list=PLDDAxmBan0BKeIxuYWjMPBWGXDqNRaW5S'
}

In [3]:
url = "https://www.youtube.com/watch?v=TI8nSJ9-C3c"
ydl_opts = {
    'format': 'bestaudio/best',
    'ignoreerrors': True,
    'outtmpl': '%(title)s.%(ext)s',
    'playlistend': 1,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
        'preferredquality': '192',
    }],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)

In [5]:
response = requests.get('https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S')

In [6]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [8]:
link_sfxs = ([a['href'] for a in soup.find_all('a', class_='pl-video-title-link')])

In [9]:
url = 'https://www.youtube.com' + link_sfxs[0]
ydl.download([url])

[youtube:playlist] Downloading playlist PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S - add --no-playlist to just download video ZEWGyyLiqY4
[youtube:playlist] PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S: Downloading webpage
[download] Downloading playlist: Best Country Songs of All Time - Top Country Music Videos
[youtube:playlist] playlist Best Country Songs of All Time - Top Country Music Videos: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube] ZEWGyyLiqY4: Downloading webpage
[youtube] ZEWGyyLiqY4: Downloading video info webpage
[download] Destination: Blake Shelton - God's Country (Official Music Video).webm
[download] 100% of 3.29MiB in 00:01
[ffmpeg] Destination: Blake Shelton - God's Country (Official Music Video).m4a
Deleting original file Blake Shelton - God's Country (Official Music Video).webm (pass -k to keep)
[download] Finished downloading playlist: Best Country Songs of All Time - Top Country Music Videos


0

## Broad overview
For each genre
1. Get urls (_beatifulSoup_)
For each url
 1. Download - url to m4a(_youtube_dl_)
 3. MIR - m4a to sonic features(_librosa_)
 5. Remove *.m4a

## Detailed
for each genre:
1. collect urls (_beatifulSoup_)  
for each url
 1. __Download__ - url to m4a(_youtube_dl_)
 2. __Assign filepath__(_glob_)
 3. __MIR__ - m4a to array(_librosa_)
 4. __MIR__ - array to individual segments  
 5. __For each segment__
    1. __MIR__ - array to beats, mfccs, zcr...  
    2. Append to __db__(_sql/csv/json??_)
    
 5. Remove *.m4a

EDGE CASE - error handling

In [44]:
path = glob('*.m4a')[0]

In [51]:
genre = 'country'

In [31]:
ts, sr = librosa.load(
    path=path,
    sr=None,
    duration=240
)

In [53]:
song_rows = []
for y in np.array_split(ts,20):
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rmse(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
    
    seg_features = {
        'song': path,
        'genre': genre,
        'tempo' : tempo,
        'beats' : beats.shape[0],
        'chroma_stft' : chroma_stft.mean(),
        'rmse' : rmse.mean(),
        'spec_cent' : spec_cent.mean(),
        'spec_bw' : spec_bw.mean(), 
        'rolloff' : rolloff.mean(), 
        'zcr' : zcr.mean()
    }
    seg_features.update(
        {f'mfcc{i+1}': mfcc.mean() for i, mfcc in enumerate(mfccs)}
    )
    song_rows.append(seg_features)

In [55]:
pd.DataFrame(song_rows)

,song,genre,tempo,beats,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,...,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12
0,Blake Shelton - God's Country (Official Music ...,country,140.625000,23,0.422453,0.153831,1428.420836,2033.222188,2471.814040,0.022851,...,18.133806,17.023444,-5.090936,7.739393,-1.601150,1.326289,-2.314972,3.627491,0.751836,2.517607
1,Blake Shelton - God's Country (Official Music ...,country,92.213115,16,0.483301,0.223698,1494.549007,2301.968912,2717.619032,0.026674,...,0.595009,25.099446,-8.849128,17.790947,-1.484787,5.708385,4.551252,8.819864,5.783755,6.351653
2,Blake Shelton - God's Country (Official Music ...,country,140.625000,23,0.407351,0.166968,1899.749916,2535.416841,3452.699884,0.038273,...,-25.846717,29.323016,-4.578676,17.301554,-13.095617,2.006380,2.760350,-3.057746,1.419105,-1.350133
3,Blake Shelton - God's Country (Official Music ...,country,92.213115,15,0.431307,0.166841,2118.240999,2647.591646,3941.037282,0.043818,...,-34.954392,30.500148,-5.871271,22.008850,-13.779923,0.600407,5.034395,-1.823981,2.997842,-2.590728
4,Blake Shelton - God's Country (Official Music ...,country,144.230769,24,0.459472,0.223341,2346.955708,2771.294449,4344.263347,0.052973,...,-37.470596,29.120298,-9.435678,15.027147,-13.429264,10.287121,-0.741138,1.153415,-0.006705,2.818009
5,Blake Shelton - God's Country (Official Music ...,country,140.625000,24,0.455672,0.257992,2408.434435,2846.210037,4623.636422,0.052063,...,-46.716459,34.867238,-9.702425,18.369922,-11.762618,11.448158,-0.377795,5.696846,-2.816398,2.779618
6,Blake Shelton - God's Country (Official Music ...,country,140.625000,23,0.497233,0.299809,2235.007066,2804.371062,4311.465285,0.044840,...,-44.186045,38.335992,-3.993637,24.618114,-7.672434,11.407171,-2.114386,5.709991,2.435314,5.438621
7,Blake Shelton - God's Country (Official Music ...,country,92.213115,16,0.486700,0.256853,2019.066690,2652.657494,3793.770855,0.035952,...,-32.189747,36.880868,-1.889923,26.042998,-9.385550,4.854355,7.981635,2.302208,3.942792,2.858381
8,Blake Shelton - God's Country (Official Music ...,country,93.750000,16,0.521020,0.294768,2306.897556,2928.646597,4524.279710,0.043729,...,-41.277058,37.195380,-13.562176,24.187290,-9.810978,12.510420,2.476704,10.870923,3.336885,7.310211
9,Blake Shelton - God's Country (Official Music ...,country,140.625000,24,0.499901,0.310640,2229.338818,2777.884952,4357.185254,0.045379,...,-45.369452,36.330097,-10.506350,19.815379,-7.684057,12.747523,2.771379,9.276654,0.750185,7.481982


In [108]:
! rm *.m4a